<img src = "https://datasciencebocconi.github.io/Images/Other/logoBocconi.png">

$\newcommand{\bb}{\boldsymbol{\beta}}$
$\newcommand{\bg}{\boldsymbol{\gamma}}$
$\DeclareMathOperator{\Gau}{\mathcal{N}}$
$\newcommand{\bphi}{\boldsymbol \phi}$
$\newcommand{\bx}{\boldsymbol{x}}$
$\newcommand{\by}{\boldsymbol{y}}$
$\newcommand{\whbb}{\widehat{\bb}}$
$\newcommand{\hf}{\hat{f}}$
$\newcommand{\tf}{\tilde{f}}$
$\newcommand{\ybar}{\overline{y}}$
$\newcommand{\E}{\mathbb{E}}$
$\newcommand{\Var}{Var}$
$\newcommand{\Cov}{Cov}$
$\newcommand{\Cor}{Cor}$

# Training module on non-linear modeling via tree-based methods 
## Topics covered:
+ Decision trees (for regression and classification)
+ Bagging
+ Random forests
+ Boosting and gradient boosting


This module introduces some fundamental concepts for effective predictive modelling based on **decision trees** and their extensions relying on esemble of multiple trees (i.e., **bagging**, **random forests** and **boosting**). The models and algorithms discussed here, combined also with linear models and regularisation, provide state-of-the-art solutions often used in practice due to their high prediction accuracy.  

As you will see, within this paradigm the features become part of the learning procedure $-$ not merely by selection, as e.g. with lasso.

**credits**: some pictures are taken from *The Elements of Statistical Learning* by Hastie, Tibshirani and Friedman (https://hastie.su.domains/ElemStatLearn/), and  *An Introduction to Statistical Learning* by James, Witten, Hastie, Tibshirani (https://www.statlearning.com/).


## Summary
**Decision trees** for regression and classification rely on partitions of the space of explanatory variables (input space) into a number of simple and non-overlapping **regions**. For each region a summary output value is then generated using data in that region, and such value will be used to predict the response for all those units (present and future) falling in that region.   

As we will see through extensive Python implementations, the above approach allows to account for and learn non-linear relations between the output and the inputs in a simple, yet effective, way. This is done through the careful identification of suitable regions via a **sequential binary splitting** strategy which, at each step, produces a new region by splitting in two one of the previously identified regions so that the new resulting partition of the training data has minimum variability, within each region, in terms of the output value to predict. Since the set of splitting rules can be summarized in a tree with branches and leaves, these approaches are known as decision tree methods.

The figure below provides an example of a **regression tree**, where we partitioned (in a greedy fashion) the input space in 3 regions and assigned an estimate of the log-salary $y$ to each of them (to be used for prediction).

+ $R_1=\{\bx : \texttt{years} < 4.5 \} \longrightarrow \hat{y}_{R_1}=\mbox{Ave}[y_i : \bx_i \in R_1]$
+ $R_2=\{\bx : \texttt{years} \geq 4.5, \texttt{hits}< 117.5  \}\longrightarrow \hat{y}_{R_2}=\mbox{Ave}[y_i : \bx_i \in R_2]$
+ $R_3=\{\bx : \texttt{years} \geq 4.5, \texttt{hits}\geq 117.5  \}\longrightarrow \hat{y}_{R_3}=\mbox{Ave}[y_i : \bx_i \in R_3]$

<img src = "https://datasciencebocconi.github.io/Images/DecisionTrees/example_tree.jpeg">

Focusing again on the above example:
+ The regions $R_1, R_2$ and $R_3$ are called **terminal nodes** or **leaves**
+ Trees are typically  drawn upside-down (the leaves are at the bottom)
+ The points along the tree where the predictor space is split are called **internal nodes**. In the example $\texttt{years} < 4.5$ and $\texttt{hits} < 117.5$ are two internal nodes

Trees are **explainable**. In the above example the input $\texttt{years}$  is the most important factor in determining the $\texttt{log-salary}$, and players with less experience earn lower salaries than more experienced players. Indeed, given that a player is less experienced, the number of $\texttt{hits}$ made in the previous year seems to play little role on the $\texttt{log-salary}$. However, among players who have been in the major leagues for five or more $\texttt{years}$, the number of $\texttt{hits}$  made in the previous year does affect $\texttt{log-salary}$.

Although basic classification and regression trees (CART) are very interpretable, such solutions can be often improved in terms of predictive accuracy. Therefore, we will also discuss **bagging**, **random forests** and **boosting**, which combine multiple trees and often lead to dramatic improvements in prediction. Their implementations are presented here using sklearn package of Python.

In [ ]:
# We load the relevant modules

%matplotlib inline
import matplotlib.pylab as plt
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn

In [ ]:
# A helper function to display the tree.
# NOTE: requires pydotplus and graphviz libraries. 
#       for MACOS/LINUX just install by typing "conda install pydotplus" and "conda install -c conda-forge python-graphviz" at the terminal
#       for Windows install by typing "conda install pydotplus" at the Anaconda Powershell Prompt 
#       for Windows and graphviz, if conda install doesn't work, then download the zip file from https://graphviz.gitlab.io/_pages/Download/Download_windows.html
#             unzip, include the 'bin' folder path in an environment variable named 'path' then do 'pip install graphviz' and 'conda install graphviz' from anaconda prompt
#             finally, also include in the environment variable 'path' the path to the Anaconda graphviz package, i.e. SomePath\Anaconda3\Library\bin\graphviz
    
from IPython.display import Image 
import pydotplus
import graphviz
def plot_tree(clf, feature_names, target_names):
    dot_data = sklearn.tree.export_graphviz(clf, out_file=None, 
                             feature_names=feature_names,  
                             class_names= target_names,  
                             filled=True, rounded=True,  
                             special_characters=True) 
    return pydotplus.graph_from_dot_data(dot_data).create_png() 


## Regression models with tree-based features

To work our way towards tree-based models it is useful to rewrite these constructions as a linear regression model with **tree-based features** 

Recall again the example below, and let $x_1$ and $x_2$ denote $\texttt{years}$ and $\texttt{hits}$, respectively. 

<img src = "https://datasciencebocconi.github.io/Images/DecisionTrees/example_tree.jpeg">

Then, the above tree-based model can be expressed as

$$f(x_1,x_2) =  5.11 \cdot 1[(x_1,x_2) \in R_1] + 6.00 \cdot 1[(x_1,x_2) \in R_2] + 6.74\cdot 1[(x_1,x_2) \in R_3]$$

More generally, if we have $p$ inputs $\bx=(x_1, \ldots, x_p)$, then

$$f(\bx;\bb) =  \sum_{m=1}^M \beta_m 1[\bx \in R_m] $$

where the $R_1, \ldots, R_M$ are disjoint regions of the input space defined by a binary tree, say splitting each variable on the median. Below, there is a more generic partitioned space of 2 variables.

<img src = "https://datasciencebocconi.github.io/Images/DecisionTrees/example_tree_2.png">

As you can notice from the figure above, the assumption here is that the **regression function** $f(\bx;\bb)$ is constant in each of these regions, hence the data are split in homogenous subgroups. We are basically fitting a **piecewise constant function**.

If the regions were known, it is trivial to fit this regression model to data: Each $\beta_m$ is estimated from the training that fall in the subregion alone, and it is a simple instance of a linear model. 

**However**, also the regions $R_1, \ldots, R_M$ are unknown and need to be 
estimated. This raises, at least, two main problems with this approach:

1. **Computational**: The number of possible regions (and, hence, different partitions) of the input space we should explore is huge. For instance, even in the simple case in which we have $p$ predictors and each is split in its median, this creates $2^p$ regions. Therefore we quickly end up with massive number of features $-$ and will overfit unless we take precautions. 
2. **Predictive**: In classical Machine Learning settings, we often have a large number $p$ of inputs and it typically turns out that a few, or even most, are not useful for prediction, hence we should not be subdiving the population according to these variables. On the other hand, for those that are relevant for prediction, there is no good reason why the median, or any preselected quantile, provides a good split. We would like to learn from the data itself where to split each variable (if used at all in the model); and maybe in order to get a good predictive model we might have to split some variables several times. Which now feeds back into the "computational" problem since the  potential regions might be much larger than $2^p$.

### Tree based models: a first illustration

Tree-based models, and the associated learning algorithms for fitting them to data, address the two challenges discussed above and result in a non-linear model where the variables to be split and the split points are learned from data (i.e., we learn not only $\beta_1, \ldots, \beta_M$ from the data, but also the regions $R_1, \ldots, R_M$). 

Before we discuss more in detail the methods and algorithms to address the above goal, let's immediatelly implement one such algorithm to the **spam** training data. The objective of this particular example is to classify emails as 'spam' or 'no spam', based on features that are basically the occurrence of certain words or expressions.

What we are doing below is **bad practice**: never run a function you do not know what it does $-$ especially when it has many options.

In [ ]:
# Load the spam dataset
dataset = pd.read_csv("https://datasciencebocconi.github.io/Data/spam_small_train.csv")
X = dataset.drop('class', axis = 1)
y = dataset["class"]

# to print stats
feature_names = X.columns
class_labels = ["email", "spam"]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X,y)

In [ ]:
# plot the tree
Image(plot_tree(model, feature_names, class_labels))

Note the **complexity** of this tree. 


+ Orange colours mean output 'no spam'. 
+ Blue colours mean 'spam'. 

The latter are predominant in the right hand side of the tree. You can zoom into the figure by saving (right-clicking) and opening elsewhere.

Before we go any further, let's check the performance of the model. In particular, let us compute the **predicted probabilities** for both training and test data.

In [ ]:
# read the test data, extract the info and create predictions

spam_test = pd.read_csv("https://datasciencebocconi.github.io/Data/spam_small_test.csv")
Xtest = spam_test.drop("class",axis=1)
ytest = spam_test["class"]
pred = model.predict_proba(X)
test_pred = model.predict_proba(Xtest)

In [ ]:
plt.plot(pred[:,1],"o")

In [ ]:
plt.plot(test_pred[:,1],"o")

We observe that the model predicts 'spam' or 'no spam' with apparently high confidence on its results (probabilities close to 0 or 1, few points in between). This is a bit suspicious and possibility caused by **overfitting**. Therefore, it is a good idea to explore more in detail **predictive accuracy**.

### AUC recap

A useful example (defaults on credit cards) for why **ROC** curve is important.

|  | True NO | True YES |  Total
| --- | --- | --- | ---
| **Predicted NO** | 9644 | 252 | 9896
| **Predicted YES** | 23 | 81 | 104
| **Total** | 9667 | 333 | 10000

If we classified using just the prior (always class No in this case) we would make $333/10000 = 3.33\%$ errors. Moreover, of the true No, we make $23/9667 = 0.2\%$ errors. Of the true Yes, we make $252/333 = 75.7\%$.

Recall that, for binary classification models, to check performance we use the Area Under the Curve (**AUC**), considering the **ROC** (Receiver Operating Charateristic) curve. 

Let us study the **AUC for both training and test data**.

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(np.array(y), np.array(pred[:,1]))
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name='example estimator')
display.plot()
plt.show()

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(np.array(ytest), np.array(test_pred[:,1]))
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name='example estimator')
display.plot()
plt.show()

## Understanding the model and the algorithm

From the Hastie et al. book, here is an example of the **binary tree** representation of the tree-based model and the corresponding **partition of input space** into subregions, as well as the induced **regression function** $f(\bx;\bb)$.

<img src = "https://datasciencebocconi.github.io/Images/DecisionTrees/example_tree_2.png">

In this construction the model and the algorithms are pretty much interweaved. The loss function can be used pretty much as in our previous supervised learning approaches. However, here it is not entirely straightforward to define an algorithm that tries to minimize the loss function, since in this case we not only need to estimate $\bb$ but also a potentially complex partition of the input space in regions $R_1, \ldots, R_M$. More formally, in the regression context under the classical residual sum of squares loss, we look for

$$\mbox{argmin}_{\beta_1, \ldots, \beta_M; R_1, \ldots, R_M}\left\{ \sum_{i=1}^n\left(y_i- \sum_{m=1}^M \beta_m 1[\bx_i \in R_m]\right)^2\right\}$$

The above optimization problem is  **non-convex**. Hence, rather than parameterising the regions, one typically uses an algorithm that then dictates the regions. 

In particular, we can use **recursive binary splitting**: A top–down (begins at the top of the tree and then considers successive binary splits to grow the tree one region at-a-time) greedy (the best split is made at that each step, rather than picking a split that will lead to a better tree in some future step) routine.

More specifically, recursive binary splitting uses a heuristic **forward search**, which is somehow reminiscent of the one used for variable selection in regression.
+ To grow the tree one region more, all possible variables are considered. For each variable the optimal cutoff point is found and then the variable that achieves largest decrease in loss function is chosen to determine the next level
+ This is heuristic but very fast to implement

Below you can find an illustrative example of this algorithm.

<img src = "https://datasciencebocconi.github.io/Images/DecisionTrees/binary_splitting_example.png">

This approach is by no means immune to overfitting $-$ and we have seen this in the **spam** example.
+ One approach is to bound the depth
+ Another is to use a **backward search** after growing the tree at its maximum depth. This is done by pruning back the tree and remove branches in order to obtain a good balance between bias and variance. The backward criterion is effectively a penalized likelihood criterion based again on a **regularization parameter**. 
+ As usual, a predictive measure is used to estimate the regularizing parameter (depth/backward parameter). 

### Behind the scenes

+ Non-convex optimisation and heuristics
  + forward-backward search
  +  model fit criteria for tree-based models (e.g. regression and classification)
  +  model complexity criteria for tree-based models; regularisation parameter
+ Variable selection and importance
+ Categorical predictors and missing data

Let's revisit the sklearn class DecisionTree

## Complexity, accuracy and overfitting
Here we study more in detail the **overfitting** risk in relation to some quantities defining the complexity of the tree. In particular, allowing a growing **number of final leaves** in our tree (i.e., more and more regions), we increase the accuracy on training data, but not necessarily on new (test) data $-$ which would be actually our final goal. 

By grid-searching the hyperparameter 'max_leaf_nodes', we can find optimum value to avoid overfitting.

In [ ]:
# some sklearn tools
from sklearn.metrics import accuracy_score

# keep the results in a list
complexity_value = []
test_accuracy = []
train_accuracy = []

# loop through possible values of max_leaf_nodes
for max_leaf_nodes in range (2, 200): 
    model  = DecisionTreeClassifier(criterion = "entropy", max_leaf_nodes=max_leaf_nodes) 
    model.fit(X, y)
    
    # predict both on train and test set
    y_pred = model.predict(Xtest)
    y_pred_train = model.predict(X)
    
    # store the data to be used for plotting
    train_accuracy.append(accuracy_score(y, y_pred_train))
    test_accuracy.append(accuracy_score(ytest, y_pred))
    complexity_value.append(max_leaf_nodes)
    
print ("Best accuracy: ", max(test_accuracy), 'at max_leaf_nodes = ', complexity_value[np.argmax(test_accuracy)])
plt.plot(complexity_value, train_accuracy, label='Train accuracy')
plt.plot(complexity_value, test_accuracy, label='Test accuracy')
plt.xlabel("complexity")
plt.ylabel("Accuracy")
plt.legend()

## Implicit variable selection and scoring

The model and algorithm effectively do a **variable selection** and **scoring**. More specially, we expect that:

+ input variables which are **not predictive** drop out (i.e., are never chosen for split)
+ input variables which are **predictive** are used to build the tree and, among them, the most important would lead to larger reduction in the loss function when used for a split.

We can, therefore, monitor for each variable the **total reduction in the loss function** achived by splits in the tree which use that variable, and then **rank variables** according to this measure. Although, much like for lasso one should not take very literally the variable selection aspect of these algorithms. 

In [ ]:
model  = DecisionTreeClassifier(max_leaf_nodes=20) 
model.fit(X, y)
important_features = pd.DataFrame(model.feature_importances_/model.feature_importances_.max() ,index=X.columns, columns=['importance'])
# it is common to normalize by the importance of the highest
important_features.sort_values('importance', ascending=False)

In [ ]:
# Lets see the tree too

Image(plot_tree(model, feature_names, class_labels))
# do right click save image as and open outside the notebook to see details

## Some details on classification trees

So far, we have presented the details of trees with a main focus on regression (i.e., when $y$ is quantitative). The application to the spam dataset is instead a situation in which $y$ is qualitative (binary in this case). Therefore, before moving to bagging and random forests, it is worth discussing some details on **classification trees**.

**Classification trees** are conceptually very similar to regression trees, with the only difference that the variable $y$ of interest is **qualitative**. Hence, we  cannot use the sample mean of $y$ to make predictions and it is not possible to use the MSE as a loss function to perform the splits or to choose among trees.

Luckily, the modification required to the regression trees framework are only minor. Compared to regression trees

+ $\hat{y}_{R_m}$ coincides with the  most commonly occurring class of training observations in $R_m$, for each $m=1, \ldots, M$
+ MSE is replaced by  **miss-classification rate**, **Gini index** or **cross-entropy**

After these replacements have been made, tree building and pruning remain the same.

Note that predicting within each region  ${R_m}, \ m=1, \ldots, M$ with the most commonly occurring class is the same as choosing the label having the highest relative frequency (computed from the training observations in $R_m$). In particular, 
$$\hat{y}_{R_m}=\mbox{argmax}_l (\hat{p}_{ml}), \quad \mbox{with} \quad \hat{p}_{ml}=\frac{1}{n_m} \sum_{i : \bx_i \in R_m} 1(y_i=l), \ l=1, \ldots, L,$$
where $\hat{p}_{ml}$ is the proportion of class $l$ observations in  region  $R_m$.

Based on the above prediction strategy, a natural way to replace the $\mbox{MSE}_m$ of the generic region ${R_m}, \ m=1, \ldots, M$, is to consider the  **miss-classification rate**
$$\mbox{E}_m=\frac{1}{n_m} \sum_{i : \bx_i \in R_m} 1[y_i \neq \mbox{argmax}_l (\hat{p}_{ml})]=1-\mbox{max}_l(\hat{p}_{ml}).$$

Using $\mbox{E}_m$, instead of the $\mbox{MSE}_m$, for each $m=1, \ldots, M$, we can easily perform tree building via recursive binary splitting and cost complexity pruning as in regression trees. **However**, it turns out that $\mbox{E}_m$ is not sufficiently sensitive for tree-growing and, in practice, the **Gini index** or the **cross-entropy** are better.

To understand this, consider this **example**: in a 2-class study with $400$ units per class [denoted by $(400, 400)$], suppose that one split created nodes $(300, 100)$ and $(100, 300)$, while the other created nodes $(200, 400)$ and $(200, 0)$. Both splits produce  miss-classification rates of 0.25, but the second provides a pure node and is probably preferable. Both the **Gini index** and the **cross-entropy** are lower for the second split.

Both **Gini index** and the **cross-entropy** will take a value close to zero if the region $R_m$ contains mostly observations from a single class, thus providing a **measure of purity**. For these reasons they are preferable when building and pruning trees.


## From trees to bagging to random forests


What motivates **bagging** and **random forests** is actually an issue of basic decision trees, which affects the predictive performance of these procedures. This issue is the **high variance** ("instability") of basic decision trees, which is massively reduced in bagging and random forests via **ensembling**.

Therefore, let us first study the stability of trees to better appreciate the subsequent developments we will see in bagging and random forests.

In [ ]:
from sklearn.model_selection import train_test_split

test_accuracy_argmax = [] # the maximal test accuracy achieved for each split
importance_char = [] # the variable char_! importance

for bootsam in np.arange(100):
    # split randomly dataset; do not fix the seed to see variation
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    #
    # First search for depth 
    test_accuracy = []
    complexity_value = []
    for max_leaf_nodes in np.arange (5, 40): 
        model  = DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes) 
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        test_accuracy.append(accuracy_score(y_test, y_pred))
        complexity_value.append(max_leaf_nodes)
    test_accuracy_argmax.append(complexity_value[np.argmax(test_accuracy)])
    #
    # print("Optimum max leaf {complexity_value[np.argmax(test_accuracy)]}")
    # Then find and store the relative importance of fare for the chosen tree
    
    model  = DecisionTreeClassifier(max_leaf_nodes=complexity_value[np.argmax(test_accuracy)]) 
    model.fit(X_train, y_train)
    important_features = pd.DataFrame(model.feature_importances_/model.feature_importances_.max() ,index=X.columns, columns=['importance'])
    importance_char.append(important_features.loc["char_freq_!",:].values[0])
    
# Print the results in a convenient manner
result =pd.DataFrame(test_accuracy_argmax,columns=["depth"])
result["score_char"] = importance_char
result.plot(x="depth",y="score_char",kind="scatter")

result.describe()

The optimum number of leaves oscillates, so optimum model parameterization can differ depending on the subset of data. 

**Take home message**: Small variations in the training data yield to large changes in the estimated tree and, as a direct consequence, substatial variability in the predictions. **Trees have high variance**. Bagging and random forests address this problem.

## Bagging

The earlier experiment in which we have grown a tree-based model on different subsets of the data revealed an important aspect of tree-based models, that of **instability**: small changes in the data (e.g. removing 20% of observations) causes large changes in the learned model. This implies **high variance** in the predictions and, recalling the bias-variance tradeoff, it reduces predictive accuracy for test data.

Here is another example with some simulated data from Hastie et al. book


<img src="https://datasciencebocconi.github.io/Images/DecisionTrees/bagging.png"> 

**Bootstrap aggregation**, or **bagging**, is a general-purpose procedure for reducing the variance of a statistical learning method. We present it here because it is very useful and frequently used in the context of decision trees (but is more general).

**Bagging** (and also random forests as we will see later on) relies on a very simple result from probability.

If we have a sample of $B$ variables (e.g., the predictions $\hat{f}_1(\bx)=\hat{f}_1(\bx;\hat{\bb}_1), \ldots, \hat{f}_B(\bx)=\hat{f}_B(\bx;\hat{\bb}_B)$ provided by $B$ trees trained on different datasets) with variance  $\sigma^2$ and positive pairwise correlation $\rho$, then, the average $\bar{f}(\bx)$ of these variables has variance $\sigma^2(1-\rho)/B+\rho \sigma^2$. More formally

$$\mbox{var}(\bar{f}(\bx))=\mbox{var}[(\sum\nolimits_{b=1}^B \hat{f}_b(\bx))/B]= \sigma^2(1-\rho)/B+\rho \sigma^2$$

In other words, **averaging over a set of observations can reduce variance**. Therefore, it may be possible to reduce the variance by averaging across predictions made by trees estimated on different training sets. 

+ **Problem**: we have only one training set and not many. 
+ **Solution**: we can use **bootstrap** to produce, from the single training dataset, several training datasets.

Below you can see a graphical representation of the **bootstrap** idea (from the book *An Introduction to Statistical Learning* by James, Witten, Hastie, Tibshirani).

<img src="https://datasciencebocconi.github.io/Images/DecisionTrees/bootstrap.png">

This is actually what **bagging** does. B(ootstrap) AGG(regation) ING tries to estimate the population-averaged estimator by boostrapping the procedure and averaging across datasets. The resultant learned function is not anymore a decision tree, but a linear combination of trees. This makes bagging an ensemble method $-$ effectively a **model averaging** approach. 

This is what happens on the same simulated dataset of Hastie et al. book.

<img src="https://datasciencebocconi.github.io/Images/DecisionTrees/bagged_trees_error.png">

### A graphical illustration of bagging
Below you can find a graphical illustration of bagging in the context of **regression**.
<img src="https://tommasorigon.github.io/DataScienceCourse/Images/DecisionTrees/regression_bag.png"> 

As mentioned previously, in general, we do not have multiple training datasets, but we can **bootstrap** from the training data and, for each of them, estimate a tree $T_b$ which outputs a prediction $\hat{f}_b(\bx)$ for $y$ at $\bx$. Clearly, every sample  $b=1, \ldots, B$ provides a different $\hat{f}_b(\bx)$ and, hence, we need to aggregate by averaging, namely
$$\bar{f}(\bx)=\sum_{b=1}^B\hat{f}_{b}(\bx)/B.$$


As illustrated below, bagging of **classification** trees works in a similar manner after minor changes.
<img src="https://datasciencebocconi.github.io/Images/DecisionTrees/classification_bag.png"> 

In the classification setting we generate again $B$ **bootstrap** samples from the training data and, for each of them, we estimate $T_b$ which outputs a predicted class for every $\bx$ via $\hat{f}_{b}(\bx)$. Clearly, every sample $b=1, \ldots, B$ provides a possibly different predicted class and, hence, we need to aggregate. This is done via **majority vote**: i.e. $\bar{f}(\bx)$ provides the most commonly occurring class among the $B$ predictions. 


## Random forests

To introduce **random forests** recall that if we have a sample of $B$ variables (e.g., the predictions $\hat{f}_1(\bx), \ldots, \hat{f}_B(\bx)$ provided by $B$ trees trained on different datasets) with variance  $\sigma^2$ and positive pairwise correlation $\rho$, then

$$\mbox{var}(\bar{f}(\bx))=\mbox{var}[(\sum\nolimits_{b=1}^B \hat{f}_b(\bx))/B]= \sigma^2(1-\rho)/B+\rho \sigma^2.$$

As is clear from the above equation, bagging helps in reducing the first summand, but not the second! In fact, the tree-based estimators from each bootstrap sample are correlated with each other. If they are significantly positively correlated (i.e., $\rho$ is close to 1), then bagging has little effect and $\mbox{var}(\bar{f}(\bx)) \approx \sigma^2$ (same as for a single tree).

The above argument motivates a modification of bagging which tries to produce tree-based estimators with as little correlation between them as possible. This is what **random forests** do. 

More specifically, random forests try to decorrelate the trees grown via bagging from different boostrap samples by including some source of randomness in their growing process (which does not increase the bias too much). Algorithmically, this is done via a minor change to the common forward-backward heuristic optimisation algorithm used for growining trees. At forward steps, instead of considering all variables as candidates for splitting, choose a **random subset** of $k < p$ inputs each time and consider only those. The resulting trees, one for each bootstrap sample, are averaged to produce the random forest estimator. 


### Bagging and random forests in sklearn

Note that random forests are a generalization of bagging which include the latter as a special case (i.e., when $k=p$). Hence, let us focus on implementing **bagging** as a special case of random forests in sklearn considering again the **spam** dataset.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

np.random.seed(31415) # impose random seed for reproducibility

dataset = pd.read_csv("https://datasciencebocconi.github.io/Data/spam_small_train.csv")
X = dataset.drop(['class'], axis = 1)
y = dataset["class"]

# to print stats
feature_names = X.columns
class_labels = ["email", "spam"]

# Bagging
bagging = RandomForestClassifier(n_estimators=20,max_features=11)
scores = cross_val_score(bagging, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

We improve relative to a single tree, but there might be room for additional improvements by including randomness in the tree construction (to decorrelate trees) via the **random forests** idea. Let's see this using also the function *GridSearchCV*.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Set the grid of parameters to explore
param_grid = {
    'bootstrap': [True],
    'max_features': [1,3,5,11],
    'n_estimators': [10,20]
}

# Create a based model
rf = RandomForestClassifier(random_state=0) 
# this is for reproducibility - check documentation!

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X, y)

# Check the optimal configuration of parameters
grid_search.best_params_

In [ ]:
forest_new = RandomForestClassifier(n_estimators=20,max_features=5)
scores = cross_val_score(forest_new, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

importances = forest_new.fit(X,y).feature_importances_
important_features = pd.Series(data=importances/importances.max() ,index=feature_names)
important_features.sort_values(ascending=False)

### Excercise 1
As an exercise, check how accuracy is affected when dropping the features that are less important (i.e., importance less than 0.05). Increase the reported decimals if needed.

In [ ]:
# Your code here


**Some comments** before moving to boosting.

+ Variable importance measures can be obtained also in random forests and bagging
+ There are 2 main tuning parameters in random forests: $k$ and $B$. In general $k$ is  fixed at some value close to $\sqrt{p}$ or is selected via cross-validation. Regarding $B$, it can be shown that the global error converges to a lower bound when $B$ grows. Hence, $B$ is typically fixed at a conservative value.

## Boosting 

Also **boosting** combines multiple trees. However,  unlike bagging and random forests, these trees are grown sequentially using weighted versions of the training data, with the weights depending on the errors made by the previous trees. **Boosting learns from its errors!**. 

Boosting is general (it can be applied to many ML methods), but here we focus on regression and classification trees, since combining boosting with tree structures often yield the best results in practice.
 
Unlike fitting a single large decision tree (which amounts to  fitting the data hard and potentially overfitting) the boosting approach instead **learns slowly from its errors**. 

+ Learning from errors has a main impact in terms of **bias reduction**. 
+ Doing it slowly, allows to **avoid rapid overfitting** and, hence, excessive variance.

Boosting has links to a number of **key ideas** in machine learning:

+ Bagging, by doing various passes through the data and combining the resulting estimators.
+ Forward search for heuristic optimisation, as it turns out that a popular implementation does exactly that.
+ Linear models with regularisation, as it turns out it is a linear combination of features, partially linear in the parameters, with a regulariser for complexity. 

Boosting combines all the above general ideas.

There are many ways to arrive to boosting algorithms and to some extent it is a matter of taste which seems more intuitive. Here we follow the derivation of boosting via forward stegewise additive modelling, along the lines in the book by Hastie, T., Tibshirani, R., Friedman, J., 2009. Elements of Statistical Learning.

### Boosting and forward stegewise additive modelling

Consider a so-called **weak learner**, which is a low bias-high variance predictive model, $b(\bx,\bg_b)$. From those we will build an **additive meta-learner** 

$$f_B(\bx) = \sum_{b=1}^B \beta_b b(\bx,\gamma_b) 
             = f_{M-1}(\bx) + \beta_M b(\bx,\gamma_B)$$
             
by learning in a **forward-search** way the parameters $\beta_b,\gamma_b$.              

The following are some standard cases:



1. Gaussian regression with general weak learner

    The loss for data point $i$ becomes 

    $$( y_i - f_B(\bx_i) )^2  = (r_i - \beta_B b(\bx,\gamma_B))^2$$

    for $r_i = y_i - f_{B-1} (\bx_i)$ the residual, hence fit the weak learner on the last residuals. For every model for which fitting the weak learner on the data is easy, you can trivially apply boosting. 

2.  Classification with weak learners in {-1,+1} and exponential loss - **AdaBoost**

      weak learner $b(\bx,\gamma_B)$ should return ${+1,-1}$, y should be binary (coded also +1,-1) and the loss for data point i should be:

    $$\exp(- y_i f_B(\bx_i)) = w_i \exp(-y_i \beta_B b(\bx,\gamma_B))$$ 
    
    for $w_i = \exp(-y_i f_{B-1}(\bx_i))$, hence we obtain, relative to fitting the weak learner only, a weighted loss function at each step of the boosting algorithm.
    
    Clever but simple manipulation yields the original AdaBoost algorithm which was derived in a different way 
      - see p.338 - 339 of Elements of Statistical Learning
 
    This loss function is closely related to **logistic link**
 
    An example of such a weak learner: a tree with 2 leaves and +1,-1 output (instead of local means) - a **stump**. 

3. Gradient boosting

    For squared loss function learning the additive weak learner expansion is easy as we saw earlier. 

    For other loss functions we can take advantage of this simplicity by computing the negative gradient of the loss function for each data point with respect to the function we wish to learn, evaluated at the current estimate

    $$- d L(y_, f(\bx_i))/d f(\bx_i) |_{f=f_{B-1}}$$

    When $L(y,f) = (y-f)^2$ this is exactly the regression residual $y-f_B$. For others (e.g. logistic link) it will be something different. The idea then is to fit a tree regression model (weak learner) on these "residuals"

<img src="https://datasciencebocconi.github.io/Images/DecisionTrees/gdb.png"> 

#### Other extensions

+ **Xgboost**: Extreme Gradient Boosting (**XGBoost**): Similar to gradient boosting but each iteration solves the gradient search in one single step using Taylor expansion of the gradient formula. On the other hand, it uses a more regularized model formalization to control over-fitting (l1, l2 parameters), which typically implies better performance.

#### Caveats
The risk of overfitting on those models has to be treated by using some **regularization parameters** (e.g. to limit the number of leaves) and/or using cross validation.

In particular, there is an issue called over-specialization, which means that trees added at later iterations tend to impact the prediction of only a few observations, so that their contribution towards the rest of the dataset becomes negligible. This is only addressed by Extreme Gradient Boosting model, by using regularization.


Lets see **boosting** in action for the **spam dataset**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

np.random.seed(3123) # impose random seed for reproducibility

tree = GradientBoostingClassifier(n_estimators=50)
scores = cross_val_score(tree, X, y, cv=10,  scoring='roc_auc')
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

importance = tree.fit(X,y).feature_importances_
important_features = pd.Series(data=importance/importance.max() ,index=feature_names)
important_features.sort_values(ascending=False)

### Exercise 2
As an exercise, check how accuracy is affected when dropping the features that are less important (i.e. importance less than 0.05). Increase the reported decimals if needed.

In [ ]:
# Your code here


## Some hints for practitioners

### Models
#### CART (single tree)
Use *sklearn* function $DecisionTreeClassifier$ for classification, or $DecisionTreeRegressor$ for regression. These models are simpler to understand and explore but typically yield worse results than the ones based on 'multiple tree' approach. Some interesting parameters  to tune (for classification) are:
+ *max_depth*: Number of levels in a tree
+ *min_samples_split*: Minimum number of samples left to try a new split
+ *min_samples_leaf*: Minimum number of samples allowed in a leaf
+ *min_impurity_decrease*: Don't accept the split if we don't reach a minimum decrease in impurity
+ *min_impurity_split*: Early stop criteria, don't split if mininum impurity has been reached
+ *class_weight*: Use 'balanced' if classes are unbalanced.


#### Random forest
Use *sklearn* function $RandomForestClassifier$ for classification, or $RandomForestRegressor$ for regression. Main parameters are:
+ *n_estimators*: Number of trees to build
+ *max_features*: Number of features to consider when looking for the best split. If less than 100%, this intruduces stochasticity.
+ *boostrap*: Keep default 'True' to allow boostrapping the sample for each tree, useful to prevent overfitting
+ *oob_score*: If True, yields a proxy for expected accuracy, using *out of bag* observations per tree.
+ *n_jobs*: Number of CPU cores to be used (parallelization).
+ *max_depth*, *min_samples_split*, *min_impurity_decrease*, *min_impurity_split*, *class_weight*: Similar to CART


#### AdaBoost
Use *sklearn* function $AdaBoostClassifier$ for classification, or $AdaBoostRegressor$ for regression. Main parameters are:
+ *n_estimators*: Number of trees to build
+ *learning_rate*: Intensity of the re-weighting (boosting) each time we build a new tree.
+ *base_estimator*: Weak learner, by default 'DecisionTreeClassifier(max_depth=1)', but one can for instance increase depth to check if this helps.

#### GradientBoosting (GBM)
Use *sklearn* function $GradientBoostingClassifier$ for classification, or $GradientBoostingRegressor$ for regression. Main parameters are:
+ *n_estimators*: Number of trees to build
+ *subsample*: Percentage of samples to use for every tree.
+ *learning_rate*: similar to AdaBoost
+ *max_features*,*max_depth*, *min_samples_split*, *min_impurity_decrease*, *min_impurity_split*, *class_weight*: Similar to random forest

#### Extreme Gradient Boosting (XGBoost)
Use *xgboost* package and follow documentation at https://xgboost.readthedocs.io/en/latest/python/

Main functions are $xgboost.XGBClassifier$ or $xgboost.XGBRegressor$, which are an API to be used with *sklearn*.

Main parameters are:
+ *gamma*: similar to min_impurity_decrease. Minimum loss reduction required to make a further partition on a leaf node of the tree.
+ *colsample_bytree*:  Percentage of features to consider when constructing each tree.
+ *reg_alpha*:  L1 regularization term on weights to control overfitting
+ *reg_lambda*: L2 regularization term on weights to control overtiffing
+ *n_estimators*,*subsample*, *learning_rate*,*max_depth*: Similar to Gradient Boosting. 

#### Other MART (Multiple Additive Regression Trees) models:
+ *ligthgbm* package: Computationally faster than GBM, https://lightgbm.readthedocs.io/en/latest/Python-Intro.html
+ *catboost* package: Computationally faster than GBM, able to manage categorical variables, allows using GPU, https://github.com/catboost/catboost and https://catboost.ai/
+ Extremely Randomized Trees: Similar to random forest but optimization of splits is based on discrete random thresholds instead of exploring the entire space, so it's faster (and sometimes more accurate since is less prone to overfit). Use *sklearn* function $ExtraTreesClassifier$ or *ExtraTreesRegressor*

### Hyperparameter optimization
Use *sklearn* functions such as $GridSearchCV$ or $RandomizedSearchCV$. See example above in random forest section.

## References

Friedmann, J., 2001. *Greedy Function Approximation: A Gradient Boosting Machine*. The Annals of Statistics 29(5) https://statweb.stanford.edu/~jhf/ftp/trebst.pdf

Hastie, T., Tibshirani, R., Friedman, J., 2009. *Elements of Statistical Learning*. 2nd Edition. Chapters 4.5, 9.1, 9.2, 10 and 15.  https://web.stanford.edu/~hastie/ElemStatLearn/

Chen, T., Guestrin, C., 2016. *XGBoost: A Scalable Tree Boosting System*. Proceedings at Conference of Knowledge Discovery and Data Mining, August 13-17, 2016, San Francisco, CA, USA. https://www.kdd.org/kdd2016/papers/files/rfp0697-chenAemb.pdf

XGBoost tutorials. https://xgboost.readthedocs.io/en/latest/tutorials/index.html